In [128]:
import pandas as pd
import numpy as np

In [129]:
import os
path = os.getcwd() #get working directory

td_top5 = pd.read_csv(path +'/processed_data/historical_team_data_top_5.csv')
td_top5_gs = pd.read_csv(path +'/processed_data/historical_team_data_top_5_gs.csv')
td_top5_adv = pd.read_csv(path + '/processed_data/historical_team_data_top_5_advanced.csv')
td_all = pd.read_csv(path +'/processed_data/historical_team_data.csv')

In [130]:
#drop data with NULLs... something is wrong with Charlotte Bobcat/Hornets data
td_top5 = td_top5.dropna(how='any') 
td_top5_gs = td_top5_gs.dropna(how='any')
td_top5_adv = td_top5_adv.dropna(how='any')
td_all = td_all.dropna(how='any')

In [131]:
#Remove 2017 season from models because it is incomplete
td_top5 = td_top5.loc[(td_top5['season'] != 2017)]
td_top5_gs = td_top5_gs.loc[(td_top5_gs['season'] != 2017)]
td_top5_adv = td_top5_adv.loc[(td_top5_adv['season'] != 2017)]
td_all = td_all.loc[(td_all['season'] != 2017)]

In [132]:
#model training attempt 1 - linear regression
##assumptions: 
##time series component will be ignored (assume league stats are same in 2016 compared to 2000) ... remove season
##no normalization of stats - could be useful
##will remove team name ... though it could be coded as a categorical variable since some orgs are better than others
##will do with all players
##drop w/l, will predict wp
data = td_all.drop(['season', 'team_name', 'w', 'l'], axis=1)

#define x, y for prediction
x = data.drop(['wp'], axis=1)
y = data['wp']

# Split data into training and validation set  using sklearn function
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)
print ('Number of samples in training data:',len(x_train))
print ('Number of samples in validation data:',len(x_test))

#perform linear regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression() #create regression object
regr.fit(x_train, y_train)
y_pred = regr.predict(x_train)

# The coefficients
print('Coefficients:', regr.coef_)
# The mean squared error
print("Mean squared error:", np.mean((y_pred - y_train) ** 2))
#The R^2
print("R^2:", r2_score(y_train, y_pred))

###Prediction is ok... 

Number of samples in training data: 331
Number of samples in validation data: 143
Coefficients: [ -7.07321032e-03  -5.98505312e-03  -2.11597438e-02  -1.22495890e-02
  -2.49508914e-02  -8.90342805e-03  -1.89302695e-02   1.54307287e-04
  -1.05307532e-02  -7.51362554e-03   1.63618024e-02   5.95133426e-03
   2.28618508e-02   2.38321365e-02  -1.98195601e-02  -7.55392572e-04
   2.50615069e-02  -1.37338266e-03   3.43314660e-02   7.32090064e-02
   4.35947833e-02  -2.70434813e-01  -7.99216197e-01   3.20691266e-02]
Mean squared error: 0.007942745875591828
R^2: 0.664485228015


In [133]:
#Test models against test set
##All players model
y_test_pred = regr.predict(x_test)

# The mean squared error
print("Mean squared error:", np.mean((y_test_pred - y_test) ** 2))
#The R^2
print("R^2:", r2_score(y_test, y_test_pred))

#Model is really mediocre

Mean squared error: 0.009268811781261157
R^2: 0.547555141713


In [134]:
#model training attempt 2 - linear regression
##assumptions: 
##time series component will be ignored (assume league stats are same in 2016 compared to 2000) ... remove season
##no normalization of stats - could be useful
##will remove team name ... though it could be coded as a categorical variable since some orgs are better than others
##will do with all players
##drop w/l, will predict wp
data = td_top5.drop(['season', 'team_name', 'w', 'l'], axis=1)

#define x, y for prediction
x = data.drop(['wp'], axis=1)
y = data['wp']

# Split data into training and validation set  using sklearn function
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)
print ('Number of samples in training data:',len(x_train))
print ('Number of samples in validation data:',len(x_test))

#perform linear regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr_top5 = linear_model.LinearRegression() #create regression object
regr_top5.fit(x_train, y_train)
y_pred = regr_top5.predict(x_train)

# The coefficients
print('Coefficients:', regr_top5.coef_)
# The mean squared error
print("Mean squared error:", np.mean((y_pred - y_train) ** 2))
#The R^2
print("R^2:", r2_score(y_train, y_pred))

#R^2 is worse but not an insurmountable delta (.68 vs .60)

Number of samples in training data: 331
Number of samples in validation data: 143
Coefficients: [ -8.88367954e-02  -8.40084161e-02  -4.73056663e-02   4.83227776e-02
  -4.31502373e-02   7.39768478e-02  -6.43107657e-02  -2.30102464e-04
  -1.44083928e-02  -5.31555808e-03   7.64141891e-03   1.10087105e-02
   4.18921855e-02   4.16576717e-02  -4.69101733e-02  -1.02853961e-02
   7.83910241e-02   7.62850384e-04   8.89900169e-01  -8.61672030e-02
  -1.32828404e+00  -4.45236647e-01   1.26639390e+00   1.20284616e-02]
Mean squared error: 0.010107860052115131
R^2: 0.573027210771


In [135]:
#Test top 5 player model against test set
y_test_pred = regr_top5.predict(x_test)

# The mean squared error
print("Mean squared error:", np.mean((y_test_pred - y_test) ** 2))
#The R^2
print("R^2:", r2_score(y_test, y_test_pred))

#Model definitely performs worse

Mean squared error: 0.01040608892909823
R^2: 0.492040453301


In [136]:
#For now will continue on to building function. However, there is clear need to improve model. Ideas:
#1. Improve features
#2. Improve top 5 player selection process
#3. Normalize data across years and/or normalize data per year to account for change in league.
#4. Try additional model types, though I suspect feature development is more critical at the moment

In [137]:
# Save models for use in app
import pickle

filename = 'all_players.sav'
pickle.dump(regr, open(path + '/models/' + filename, 'wb'))

filename = 'top_5_players.sav'
pickle.dump(regr_top5, open(path + '/models/' + filename, 'wb'))

In [138]:
#Data normalization via 'feature scaling'
data = td_top5.drop(['season', 'team_name', 'w', 'l'], axis=1)
data = (data - data.min()) / (data.max() - data.min())

#define x, y for prediction
x = data.drop(['wp'], axis=1)
y = data['wp']

# Split data into training and validation set  using sklearn function
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)
print ('Number of samples in training data:',len(x_train))
print ('Number of samples in validation data:',len(x_test))

#perform linear regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr_top5 = linear_model.LinearRegression() #create regression object
regr_top5.fit(x_train, y_train)
y_pred = regr_top5.predict(x_train)

# The coefficients
print('Coefficients:', regr_top5.coef_)
# The mean squared error
print("Mean squared error:", np.mean((y_pred - y_train) ** 2))
#The R^2
print("R^2:", r2_score(y_train, y_pred))

#Apparently feature scaling does nothing to the prediction accuracy? Because everything is in the same proportion maybe?


Number of samples in training data: 331
Number of samples in validation data: 143
Coefficients: [-1.60724094 -3.36780406 -0.64035353  1.67933158 -0.92108634  3.33154147
 -1.28907357 -0.00533107 -0.16690865 -0.11000674  0.22577361  0.2550526
  0.23991588  0.32532658 -0.43961534 -0.09772759  3.87724494  0.04974511
  0.20385777 -0.05018898 -0.30687578 -0.18515484  0.26579196  0.20744323]
Mean squared error: 0.017124023933086926
R^2: 0.573027210771


In [139]:
#Data normalization via 'standard score'
data = td_top5.drop(['season', 'team_name', 'w', 'l'], axis=1)
data = (data - data.mean()) / data.std()

#define x, y for prediction
x = data.drop(['wp'], axis=1)
y = data['wp']

# Split data into training and validation set  using sklearn function
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)
print ('Number of samples in training data:',len(x_train))
print ('Number of samples in validation data:',len(x_test))

#perform linear regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr_top5 = linear_model.LinearRegression() #create regression object
regr_top5.fit(x_train, y_train)
y_pred = regr_top5.predict(x_train)

# The coefficients
print('Coefficients:', regr_top5.coef_)
# The mean squared error
print("Mean squared error:", np.mean((y_pred - y_train) ** 2))
#The R^2
print("R^2:", r2_score(y_train, y_pred))

#Apparently standard score doesn't do anything either

Number of samples in training data: 331
Number of samples in validation data: 143
Coefficients: [-1.41806094 -2.81785166 -0.48677277  1.27200593 -0.74516263  2.59100996
 -1.07987182 -0.0049512  -0.14603141 -0.08982295  0.18510309  0.19714107
  0.22375778  0.27885864 -0.3813266  -0.08426582  3.59341471  0.04167848
  0.15320221 -0.03411591 -0.21182935 -0.12832989  0.21962328  0.18310914]
Mean squared error: 0.4440924211656911
R^2: 0.573027210771


In [140]:
#Try substracting season averages for each team to see if this scaling helps prediction

data = td_top5.drop(['team_name', 'w', 'l','wp'], axis=1)
wp = td_top5['wp']
season_average = pd.read_csv(path +'/processed_data/season_average_top_5_data.csv', index_col=0)

new_data = pd.DataFrame()
for i in season_average['season'].unique():
    diff = data[data['season']==i] - season_average[season_average['season']==i].loc['mean']
    new_data = new_data.append(diff)

data = new_data.drop(['2p', '3p', 'fg','ft', 'mp', 'season'], axis=1)
data = pd.concat([wp, data], axis=1)

#define x, y for prediction
x = data.drop(['wp'], axis=1)
y = data['wp']

# Split data into training and validation set  using sklearn function
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)
print ('Number of samples in training data:',len(x_train))
print ('Number of samples in validation data:',len(x_test))

#perform linear regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr_top5 = linear_model.LinearRegression() #create regression object
regr_top5.fit(x_train, y_train)
y_pred = regr_top5.predict(x_train)

# The coefficients
print('Coefficients:', regr_top5.coef_)
# The mean squared error
print("Mean squared error:", np.mean((y_pred - y_train) ** 2))
#The R^2
print("R^2:", r2_score(y_train, y_pred))

#R^2 improved by approximately .04 for training set... seems like a meaningful improvement

Number of samples in training data: 331
Number of samples in validation data: 143
Coefficients: [  1.87206342e-01   3.21779921e-02  -8.13540014e-02   3.16474878e-02
   8.17204029e-03   9.62884729e-03   3.58647587e-02  -5.65637087e-04
   1.03926283e+00  -8.93437645e-01  -5.64834299e-02  -6.40872809e-01
  -9.18239629e-03  -1.93886235e-02  -1.73138047e-02   2.67955822e-02
   3.58231522e-02  -5.04604996e-02   7.45911232e-03]
Mean squared error: 0.008950049872914695
R^2: 0.621935034886


In [141]:
filename = 'top_5_players_sa.sav'
pickle.dump(regr_top5, open(path + '/models/' + filename, 'wb'))

In [142]:
#Test top 5 player model against test set
y_test_pred = regr_top5.predict(x_test)

# The mean squared error
print("Mean squared error:", np.mean((y_test_pred - y_test) ** 2))
#The R^2
print("R^2:", r2_score(y_test, y_test_pred))

#Model performs much better on test set (.09 R2 improvement despite for top 5)

Mean squared error: 0.008896088065509726
R^2: 0.565749159752


In [143]:
#Try substracting season averages for each team to see if this scaling helps prediction

data = td_top5_gs.drop(['team_name', 'w', 'l','wp'], axis=1)
wp = td_top5_gs['wp']
season_average = pd.read_csv(path +'/processed_data/season_average_top_5_gs_data.csv', index_col=0)

new_data = pd.DataFrame()
for i in season_average['season'].unique():
    diff = data[data['season']==i] - season_average[season_average['season']==i].loc['mean']
    new_data = new_data.append(diff)

data = new_data.drop(['2p', '3p', 'fg','ft', 'mp', 'season'], axis=1)
data = pd.concat([wp, data], axis=1)

#define x, y for prediction
x = data.drop(['wp'], axis=1)
y = data['wp']

# Split data into training and validation set  using sklearn function
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)
print ('Number of samples in training data:',len(x_train))
print ('Number of samples in validation data:',len(x_test))

#perform linear regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr_top5 = linear_model.LinearRegression() #create regression object
regr_top5.fit(x_train, y_train)
y_pred = regr_top5.predict(x_train)

# The coefficients
print('Coefficients:', regr_top5.coef_)
# The mean squared error
print("Mean squared error:", np.mean((y_pred - y_train) ** 2))
#The R^2
print("R^2:", r2_score(y_train, y_pred))

#R^2 is slightly worse in this case versus ranking by minutes played

Number of samples in training data: 331
Number of samples in validation data: 143
Coefficients: [-0.54628527 -0.00655451 -0.09792054 -0.00764593  0.01105216  0.00817982
  0.02905397  0.02015507  1.30914952 -0.38615621 -0.02177514 -0.47113393
 -0.00904341  0.0080602  -0.01331702  0.03049581  0.03361279 -0.05321272
 -0.01481774]
Mean squared error: 0.008453978925293735
R^2: 0.642889895269


In [144]:
filename = 'top_5_players_sa_gs.sav'
pickle.dump(regr_top5, open(path + '/models/' + filename, 'wb'))

In [145]:
#Test top 5 player model against test set
y_test_pred = regr_top5.predict(x_test)

# The mean squared error
print("Mean squared error:", np.mean((y_test_pred - y_test) ** 2))
#The R^2
print("R^2:", r2_score(y_test, y_test_pred))

#This model performed best against the test set... I really should be doing a cross validation method
#right now I'm definitely introducing bias by running against the test set

Mean squared error: 0.010285639592949489
R^2: 0.49792002925


In [146]:
#Advanced stats experimentation + season scaling

#data = td_top5_adv.drop(['team_name', 'w', 'l'], axis=1)

data = td_top5_adv.drop(['team_name', 'w', 'l','wp'], axis=1)
wp = td_top5_adv['wp']
season_average = pd.read_csv(path +'/processed_data/season_average_top_5_advanced_data.csv', index_col=0)

new_data = pd.DataFrame()
for i in season_average['season'].unique():
    diff = data[data['season']==i] - season_average[season_average['season']==i].loc['mean']
    new_data = new_data.append(diff)

data = new_data.drop(['2p', '3p', 'fg','ft', 'mp', 'season'], axis=1)
data = pd.concat([wp, data], axis=1)

#define x, y for prediction
x = data.drop(['wp'], axis=1)
y = data['wp']

# Split data into training and validation set  using sklearn function
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)
print ('Number of samples in training data:',len(x_train))
print ('Number of samples in validation data:',len(x_test))

#perform linear regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr_top5 = linear_model.LinearRegression() #create regression object
regr_top5.fit(x_train, y_train)
y_pred = regr_top5.predict(x_train)

# The coefficients
print('Coefficients:', regr_top5.coef_)
# The mean squared error
print("Mean squared error:", np.mean((y_pred - y_train) ** 2))
#The R^2
print("R^2:", r2_score(y_train, y_pred))

#R^2 is slightly worse in this case versus ranking by minutes played

Number of samples in training data: 331
Number of samples in validation data: 143
Coefficients: [ -1.00644066e-01   1.44007823e-03   3.16650076e-02  -1.88751780e-03
   2.10832694e-01   4.76553132e-03   3.07169218e-02  -2.57903738e-02
   3.86834577e-02  -2.76608277e-02   7.51033364e-02  -7.66928798e-02
   3.57014695e-02  -2.60734466e-02   3.39781956e-02  -1.21090156e-01
   7.41353448e-01   7.61356902e-03  -4.21758456e-01   2.58257739e-03
  -1.95060150e-01  -7.83573816e-02   6.00394241e-02  -6.55250698e-02
   1.01402387e-01  -8.80673144e-02  -1.46876517e-02  -1.02460412e-02
  -2.23236559e-02   1.14840030e-01   3.00175067e-04   5.51535323e-03
  -4.34888240e-02   7.95504922e-02   5.16930169e-02   9.10908094e-03
  -6.88011514e-02   6.46249642e+00]
Mean squared error: 0.002095800947333057
R^2: 0.911469888628


In [147]:
filename = 'top_5_players_sa_advanced.sav'
pickle.dump(regr_top5, open(path + '/models/' + filename, 'wb'))

In [148]:
#Test top 5 player model against test set
y_test_pred = regr_top5.predict(x_test)

# The mean squared error
print("Mean squared error:", np.mean((y_test_pred - y_test) ** 2))
#The R^2
print("R^2:", r2_score(y_test, y_test_pred))

#This model performed best against the test set... I really should be doing a cross validation method
#right now I'm definitely introducing bias by running against the test set

Mean squared error: 0.002600941299466077
R^2: 0.873038470796


In [149]:
#Advanced stats experimentation + w/o season scaling

data = td_top5_adv.drop(['team_name', 'w', 'l'], axis=1)

#define x, y for prediction
x = data.drop(['wp'], axis=1)
y = data['wp']

# Split data into training and validation set  using sklearn function
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)
print ('Number of samples in training data:',len(x_train))
print ('Number of samples in validation data:',len(x_test))

#perform linear regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr_top5 = linear_model.LinearRegression() #create regression object
regr_top5.fit(x_train, y_train)
y_pred = regr_top5.predict(x_train)

# The coefficients
print('Coefficients:', regr_top5.coef_)
# The mean squared error
print("Mean squared error:", np.mean((y_pred - y_train) ** 2))
#The R^2
print("R^2:", r2_score(y_train, y_pred))

#R^2 is slightly worse in this case versus ranking by minutes played

Number of samples in training data: 331
Number of samples in validation data: 143
Coefficients: [ -7.67185762e-05  -5.18520499e-02   7.68148358e-03   1.38470585e-02
  -1.86026971e-02  -2.02358932e-02   1.09625653e-02  -2.45033901e-02
   3.61900583e-04   5.92084485e-02   3.46704052e-02  -4.61232869e-02
   2.85001068e-02  -2.81579049e-02   4.08123229e-02   5.02819896e-03
  -1.07112208e-02   2.02245802e-02  -1.58911104e-03  -3.70035955e-01
   3.97806680e-02   7.74862756e+00  -4.70243806e-01  -6.18644405e+00
   4.76514672e-03  -6.66126470e-02   1.58327833e+00  -1.27389561e-01
  -7.13224053e-02  -3.43536486e-02   9.80200944e-02  -2.30888172e-02
   9.90775981e-02  -5.10355637e-02   6.02556220e-04   3.36100065e-02
   4.88765327e-02  -9.84751914e-03  -1.56825855e-02   5.53914976e+00
  -7.46765755e-02  -4.79053427e-02   6.05317071e-02   6.88370103e-03]
Mean squared error: 0.0020560168494829167
R^2: 0.913150434969


In [150]:
filename = 'top_5_players_advanced.sav'
pickle.dump(regr_top5, open(path + '/models/' + filename, 'wb'))

In [151]:
#Test top 5 player model against test set
y_test_pred = regr_top5.predict(x_test)

# The mean squared error
print("Mean squared error:", np.mean((y_test_pred - y_test) ** 2))
#The R^2
print("R^2:", r2_score(y_test, y_test_pred))

#This model performed best against the test set... I really should be doing a cross validation method
#right now I'm definitely introducing bias by running against the test set

Mean squared error: 0.0029499163151636347
R^2: 0.856003714319
